# Titanic - Machine Learning from Disaster

## Features constructs

> Создать новые переменные из Name

In [1]:
import numpy as np
import pandas as pd

## Импорт данных

In [2]:
df = pd.read_csv('data/train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Обработка данных

### Разделение на выборки 

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
target = df['Survived']
# features = df.drop(['PassengerId', 'Ticket', 'Cabin', 'Survived'], axis=1)
features = df.drop(['PassengerId', 'Ticket','Survived'], axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42)
X_train.shape, X_test.shape

((596, 9), (295, 9))

## Создание конвейера

План:

1. Численные:
 - Age - заполнение пропусков
2. Категорийный:
 - Embarked, Sex - Кодирование

In [6]:
import pandas as pd
from sklearn.compose import ColumnTransformer

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier


### Итоговый конвейер

In [49]:
def get_title(ser: pd.Series):
    dic_title = dict(Mr='Mr', Miss='Miss', Mrs='Mrs', Master='Master', Mlle='Miss', Ms='Miss', Mme='Mrs')
    ser = ser.str.extract(r'.*\, ([a-zA-Z]*)\. ')
    ser = ser.map(lambda x: dic_title.get(x, 'rare'))
    print(type(ser))
    return ser

def get_level(ser: pd.Series):
    ser = ser.str.slice(0, 1).fillna('.')
    print(type(ser))
    return pd.DataFrame(ser)

In [50]:
from sklearn.base import BaseEstimator, TransformerMixin


class IncreasePclass(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Входные данные должны быть pandas DataFrame")
        X1 = pd.DataFrame()
        X1['new_column'] = (10 / X.iloc[:, 0]) * X.iloc[:, 1]
        return X1

    def set_output(self, *, transform=None):
        pass

In [72]:
X_train.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin',
       'Embarked'],
      dtype='object')

In [71]:
from sklearn_pandas import DataFrameMapper

ValueError: Unable to configure output for DataFrameMapper(df_out=True, drop_cols=[],
                features=[('Name',
                           [FunctionTransformer(func=<function get_title at 0x000001293F47BF60>),
                            OrdinalEncoder()]),
                          ('Cabin',
                           [FunctionTransformer(func=<function get_level at 0x000001293F47B920>),
                            OrdinalEncoder()]),
                          (['Pclass', 'Fare'], IncreasePclass(),
                           {'alias': 'PFare'}),
                          (['Embarked'], OrdinalEncoder()),
                          (['Sex'], OrdinalEncoder())],
                input_df=True) because `set_output` is not available.

In [73]:
mapper = DataFrameMapper([
    ('Name', [FunctionTransformer(get_title), OrdinalEncoder()]), # name of the function
    ('Cabin', [FunctionTransformer(get_level),OrdinalEncoder()]),
    (['Pclass', 'Fare'], IncreasePclass(), {'alias': 'PFare'}),
    (['Embarked'], OrdinalEncoder()),
    (['Sex'], OrdinalEncoder()),
    (['Age'], None),
    (['SibSp'], None),
    (['Parch'], None),
], input_df=True, df_out=True, )



# pipe_mapper = Pipeline([('map_field', mapper),
#                         ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
#                         ('model', DecisionTreeClassifier(random_state=0))])
#
# pipe_columns = ColumnTransformer([('pre_proc', mapper, ['Name', 'Cabin', 'Pclass', 'Fare', 'Embarked', 'Sex'])],
#                                  remainder='passthrough').set_output(transform='pandas')

In [74]:
mapper.fit_transform(X_train)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


C:\prj\ml\TitanicML\venv\Lib\site-packages\sklearn\utils\deprecation.py:95: FutureWarning: Function tosequence is deprecated; tosequence was deprecated in 1.5 and will be removed in 1.7
  warnings.warn(msg, category=FutureWarning)


,Name,Cabin,PFare,Embarked,Sex,Age,SibSp,Parch
6,2.0,5.0,518.625000,2.0,1.0,54.0,0,0
718,2.0,0.0,51.666667,1.0,1.0,NaN,0,0
685,2.0,0.0,207.896000,0.0,1.0,25.0,1,2
73,2.0,0.0,48.180667,0.0,1.0,26.0,1,0
882,1.0,0.0,35.055667,2.0,0.0,22.0,0,0
...,...,...,...,...,...,...,...,...
106,1.0,0.0,25.500000,2.0,0.0,21.0,0,0
270,2.0,0.0,310.000000,2.0,1.0,NaN,0,0
860,2.0,0.0,47.027667,2.0,1.0,41.0,2,0
435,1.0,2.0,1200.000000,2.0,0.0,14.0,1,2


In [336]:
mapper.transformed_names_

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,51.8625,E46,S
718,3,"McEvoy, Mr. Michael",male,NaN,0,0,15.5000,NaN,Q


In [337]:
pipe_mapper.set_params(model__max_depth=6, model__max_features=5, knn__n_neighbors=6)

,Name,PFare,Cabin,Embarked,Sex,Age
6,Mr,518.625000,E,S,male,54.0
718,Mr,51.666667,.,Q,male,NaN
685,Mr,207.896000,.,C,male,25.0
73,Mr,48.180667,.,C,male,26.0
882,Miss,35.055667,.,S,female,22.0
...,...,...,...,...,...,...
106,Miss,25.500000,.,S,female,21.0
270,Mr,310.000000,.,S,male,NaN
860,Mr,47.027667,.,S,male,41.0
435,Miss,1200.000000,B,S,female,14.0


In [338]:
pipe_mapper.fit(X_train, y_train)

,Name,PFare,Cabin,Embarked,Sex,Age
709,Master,50.819333,.,C,male,NaN
439,Mr,52.500000,.,S,male,31.0
840,Mr,26.416667,.,S,male,20.0
720,Miss,165.000000,.,S,female,6.0
39,Miss,37.472333,.,C,female,14.0
...,...,...,...,...,...,...
715,Mr,25.500000,F,S,male,19.0
525,Mr,25.833333,.,Q,male,40.5
381,Miss,52.472333,.,C,female,1.0
140,Mrs,50.819333,.,C,female,NaN


In [339]:
X_train.head(2)

[('map_field',
  DataFrameMapper(df_out=True, drop_cols=[],
                  features=[('Name',
                             FunctionTransformer(func=<function get_title at 0x000001B183D31B20>)),
                            (['Pclass', 'Fare'], IncreasePclass(),
                             {'alias': 'PFare'}),
                            ('Cabin',
                             FunctionTransformer(func=<function get_level at 0x000001B183D33100>)),
                            ('Embarked', None), ('Sex', None), ('Age', None)],
                  input_df=True)),
 ('encoder',
  OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
 ('knn', KNNImputer(n_neighbors=6)),
 ('model',
  DecisionTreeClassifier(max_depth=6, max_features=5, random_state=0))]

In [ ]:
mapper.transform(X_train)

In [340]:
mapper.transform(X_test)

In [341]:
pipe_mapper.steps


0.8640939597315436

0.711864406779661

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
accuracy_score(y_train, pipe_mapper.predict(X_train))

Pipeline(steps=[('process',
                 Pipeline(steps=[('trans_increase_pclass',
                                  IncreasePclass(col1='Pclass', col2='Fare',
                                                 new_col_name='PFare')),
                                 ('preproc',
                                  ColumnTransformer(force_int_remainder_cols=False,
                                                    remainder='passthrough',
                                                    transformers=[('encoder',
                                                                   Pipeline(steps=[('regexp_title',
                                                                                    FunctionTransformer(func=<function get_title at 0x0000022CE51F9C60>,
                                                                                                        kw_args={'co...),
                                                                                   ('encoder_ord',
                                                                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                   unknown_value=nan))]),
                                                                   ['Name']),
                                                                  ('cat_preproc',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan),
                                                                   ['Embarked',
                                                                    'Sex'])])),
                                 ('imputer',
                                  Pipeline(steps=[('knn',
                                                   KNNImputer(n_neighbors=6))]))])),
                ('model',
                 DecisionTreeClassifier(max_depth=6, max_features=5,
                                        random_state=0))])

In [19]:
accuracy_score(y_test, pipe_mapper.predict(X_test))

Pipeline(steps=[('process',
                 Pipeline(steps=[('trans_increase_pclass',
                                  IncreasePclass(col1='Pclass', col2='Fare',
                                                 new_col_name='PFare')),
                                 ('preproc',
                                  ColumnTransformer(force_int_remainder_cols=False,
                                                    remainder='passthrough',
                                                    transformers=[('encoder',
                                                                   Pipeline(steps=[('regexp_title',
                                                                                    FunctionTransformer(func=<function get_title at 0x0000022CE51F9C60>,
                                                                                                        kw_args={'co...),
                                                                                   ('encoder_ord',
                                                                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                   unknown_value=nan))]),
                                                                   ['Name']),
                                                                  ('cat_preproc',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan),
                                                                   ['Embarked',
                                                                    'Sex'])])),
                                 ('imputer',
                                  Pipeline(steps=[('knn',
                                                   KNNImputer(n_neighbors=6))]))])),
                ('model',
                 DecisionTreeClassifier(max_depth=6, max_features=5,
                                        random_state=0))])

In [20]:
increase_pclass = Pipeline([('', IncreasePclass('Pclass', 'Fare', 'PFare'))])
trans_increase_pclass = IncreasePclass('Pclass', 'Fare', 'PFare')
encoder_ord = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
pipe_name = Pipeline([('regexp_title', FunctionTransformer(get_title, kw_args={'col': 'Name'})),
                      ('encoder_ord', encoder_ord), ])
pipe_cabin = Pipeline([('regexp_cabin', FunctionTransformer(get_level, kw_args={'col': 'Cabin'})),
                       ('encoder_ord', encoder_ord), ])
pipe_imputer = Pipeline([('knn', KNNImputer())])
pipe_preproc = ColumnTransformer(
    transformers=[
        # ('increase_pclass', increase_pclass, ['Pclass', 'Fare']),
        ('encoder', pipe_name, ['Name']),
        # ('encoder_cabin', pipe_cabin, ['Cabin']),
        ('cat_preproc', encoder_ord, ['Embarked', 'Sex'])],
    remainder='passthrough',
    force_int_remainder_cols=False).set_output(transform='pandas')

model = DecisionTreeClassifier(random_state=0)

pipe_process = Pipeline(
    [('trans_increase_pclass', trans_increase_pclass), ('preproc', pipe_preproc), ('imputer', pipe_imputer)])
pipe_model = Pipeline([('process', pipe_process), ('model', model)])


In [21]:
# pipe_model.set_params(model__max_depth=7, model__max_features=5, process__imputer__knn__n_neighbors=9)
#{'model__max_depth': 6, 'model__max_features': 5, 'process__imputer__knn__n_neighbors': 7}
# pipe_model.set_params(model__max_depth=6, model__max_features=4, process__imputer__knn__n_neighbors=3)
# pipe_model.set_params(model__max_depth=8, model__max_features=4, process__imputer__knn__n_neighbors=9)
pipe_model.set_params(model__max_depth=6, model__max_features=5, process__imputer__knn__n_neighbors=6)

0.8506711409395973

In [22]:
pipe_model.fit(X_train, y_train)

0.7932203389830509

In [ ]:
from sklearn.metrics import accuracy_score

In [23]:
accuracy_score(y_train, pipe_model.predict(X_train))

In [24]:
accuracy_score(y_test, pipe_model.predict(X_test))

## Поиск параметров

In [26]:

from sklearn.model_selection import GridSearchCV

{'model__max_depth': 4,
 'model__max_features': 5,
 'process__imputer__knn__n_neighbors': 3}

In [ ]:
model_params = {'model__max_depth': range(1, 11), 'model__max_features': range(4, 19),
                'process__imputer__knn__n_neighbors': range(3, 10)}
tree_grid = GridSearchCV(pipe_model, model_params, cv=5, n_jobs=-1, verbose=True)

In [ ]:
tree_grid.fit(X_train, y_train)


In [27]:
tree_grid.best_params_

Pipeline(steps=[('process',
                 Pipeline(steps=[('trans_increase_pclass',
                                  IncreasePclass(col1='Pclass', col2='Fare',
                                                 new_col_name='PFare')),
                                 ('preproc',
                                  ColumnTransformer(force_int_remainder_cols=False,
                                                    remainder='passthrough',
                                                    transformers=[('encoder',
                                                                   Pipeline(steps=[('regexp_title',
                                                                                    FunctionTransformer(func=<function get_title at 0x0000022CE51F9C60>,
                                                                                                        kw_args={'co...),
                                                                                   ('encoder_ord',
                                                                                    OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                   unknown_value=nan))]),
                                                                   ['Name']),
                                                                  ('cat_preproc',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan),
                                                                   ['Embarked',
                                                                    'Sex'])])),
                                 ('imputer',
                                  Pipeline(steps=[('knn',
                                                   KNNImputer(n_neighbors=3))]))])),
                ('model',
                 DecisionTreeClassifier(max_depth=4, max_features=5,
                                        random_state=0))])

### Метрики обучения

> Скор на трейне : 0.979
> Скор на валиде : 0.76
> Вывод достигли переобучение

In [30]:
pipe_model.set_params(model__max_depth=4, model__max_features=5, process__imputer__knn__n_neighbors=3)
pipe_model.fit(X_train, y_train)

0.8135593220338984

In [ ]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(y_train, pipe_model.predict(X_train))

[('process',
  Pipeline(steps=[('trans_increase_pclass',
                   IncreasePclass(col1='Pclass', col2='Fare',
                                  new_col_name='PFare')),
                  ('preproc',
                   ColumnTransformer(force_int_remainder_cols=False,
                                     remainder='passthrough',
                                     transformers=[('encoder',
                                                    Pipeline(steps=[('regexp_title',
                                                                     FunctionTransformer(func=<function get_title at 0x0000022CE51F9C60>,
                                                                                         kw_args={'col': 'Name'})),
                                                                    ('encoder_ord',
                                                                     OrdinalEncoder(handle_unknown='use_encoded_value',
                                                        

In [37]:
accuracy_score(y_test, pipe_model.predict(X_test))

,Name,Sex,Age,SibSp,Parch,Embarked,PFare
6,"McCarthy, Mr. Timothy J",male,54.0,0,0,S,518.625000
718,"McEvoy, Mr. Michael",male,NaN,0,0,Q,51.666667
685,"Laroche, Mr. Joseph Philippe Lemercier",male,25.0,1,2,C,207.896000
73,"Chronopoulos, Mr. Apostolos",male,26.0,1,0,C,48.180667
882,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,S,35.055667
...,...,...,...,...,...,...,...
106,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,S,25.500000
270,"Cairns, Mr. Alexander",male,NaN,0,0,S,310.000000
860,"Hansen, Mr. Claus Peter",male,41.0,2,0,S,47.027667
435,"Carter, Miss. Lucile Polk",female,14.0,1,2,S,1200.000000


### Исследование pipeline

In [67]:
pipe_model.steps

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [ ]:
trans_increase_pclass.transform(X_train)

In [ ]:
pipe_model.named_steps['model'].feature_importances_

In [41]:
X_train.columns

Максимальное влияние параметров:
1. Sex - 0.29
2. SibSp (наличие родственников) - 0.27
3. Embarked (порт назначения) - 0.20


### Кроссвалидация

In [31]:

from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(pipe_model, X_train, y_train, cv=5,
                             scoring='accuracy')

In [32]:
cv_results

## Предсказание на реальных данных

In [34]:
df_ground = pd.read_csv('data/test.csv')
# df_ground.head(2)

Successfully submitted to Titanic - Machine Learning from Disaster



  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|##########| 3.18k/3.18k [00:00<00:00, 6.58kB/s]


In [ ]:
df_ground['Survived'] = pipe_model.predict(df_ground.drop(['PassengerId', 'Ticket'], axis=1))
# df_ground.head()

In [ ]:
df_ground[['PassengerId', 'Survived']].to_csv('data/test14.csv', index=False)

In [ ]:
! kaggle competitions submit titanic -f .\data\test14.csv -m"pclass fare mul"

### Метрика на лидерборде

Score: 0.73923

place: 12189

После оптимизации knn

Score: 0.74880

place: 11928

Можно сказать, что улучшение качества модели существенно улучшило результат на лидерборде.




Score: 0.79665
Place: 909